This code is slightly hard to follow due to the number of ways the data was initially processed.  One could write a new script for preprocessing the data that did not include vector folders.  One could also add any organisational structure to the folders they wish.  Simply explore the layout of this 

In [2]:
import pandas as pd
import numpy as np
import pickle

In [5]:
## this is for loc_1-3 folders, for other location folders refer to lower cells

numLoc = 3
numVec = 50
numScan = 9

numVec2 = 55
numScan2 = 10


## for each location file i.e. range(1,numLoc+1)
for i in range(1,numLoc+1):
    locPath = "../raw_data/new_loc_" + str(i) + "/"
    locPath2 = "../raw_data/2_new_loc_" + str(i) + "/"
    db = 1
    ## for each vec file i.e. range(1,numVec+1)
    for j in range(1,numVec+1):
        vecPath = "vec_" + str(j) + "/"
        minRan = db
        maxRan = db+numScan
        for k in range(minRan,maxRan):
            ## print(db)
            dbPath = "db%s.csv" % k
            
            path = locPath + vecPath + dbPath
            ## "file_{}.dat".format(i)
            data = pd.read_csv(path)
        
            new = data[['MAC Address','RSSI']]
            new = new.set_index('MAC Address')
            if i == 1:
                if db == 1:
                    ## using top 8 initial signal strengths (can be changed for specific case)
                    ## these 8 will refer to 8 highest potential APs
                    ref = new[0:8]
                    reflist = ref.index.values.tolist()
                    # pickle the list for creating prdictions vectors later
                    with open('../M2_reflist.pkl', 'wb') as f:
                        pickle.dump(reflist, f)
                    ## print(reflist)
                    print(reflist)
                    ind = list(reflist)
                    ind.append('loc')
                    print(ind)
                    base = pd.DataFrame(index=ind)
                    base.to_pickle("../M2_refbase.pkl")
                    alldf = base
                    print(reflist)
                    print(alldf)
                    print(alldf.shape)
            new = new['RSSI'].to_dict()
            applist = []
            for h in reflist:
                toapp = 0
                for x,y in new.items():
                    if x == h:
                        toapp = y
                applist.append(toapp)
            applist.append(i-1)
            add = np.asarray(applist)
            if k ==1: print(add)
            alldf[('RSSI%d' % (db+((i-1)*450)))] = add
            db+=1
    db = 1
    for j in range(1,numVec2+1):
        vecPath = "vec_" + str(j) + "/"
        minRan = db
        maxRan = db+numScan2
        for k in range(minRan,maxRan):
            ## print(db)
            dbPath = "db%s.csv" % k
            
            path = locPath2 + vecPath + dbPath
            ## "file_{}.dat".format(i)
            data = pd.read_csv(path)
        
            new = data[['MAC Address','RSSI']]
            new = new.set_index('MAC Address')
            new = new['RSSI'].to_dict()
            applist = []
            for h in reflist:
                toapp = 0
                for x,y in new.items():
                    if x == h:
                        toapp = y
                applist.append(toapp)
            applist.append(i-1)
            add = np.asarray(applist)
            if k ==1: print(add)
            alldf[('RSSI%d' % (1350+db+((i-1)*550)))] = add
            db+=1
        

['C0-56-27-1B-F3-F4', 'C0-56-27-1B-9D-C5', 'C0-56-27-18-69-64', 'C0-56-27-18-69-66', 'C0-56-27-1B-F3-F5', 'C0-56-27-1B-9D-C6', 'D8-84-66-13-10-00', 'C0-56-27-18-69-65']
['C0-56-27-1B-F3-F4', 'C0-56-27-1B-9D-C5', 'C0-56-27-18-69-64', 'C0-56-27-18-69-66', 'C0-56-27-1B-F3-F5', 'C0-56-27-1B-9D-C6', 'D8-84-66-13-10-00', 'C0-56-27-18-69-65', 'loc']
['C0-56-27-1B-F3-F4', 'C0-56-27-1B-9D-C5', 'C0-56-27-18-69-64', 'C0-56-27-18-69-66', 'C0-56-27-1B-F3-F5', 'C0-56-27-1B-9D-C6', 'D8-84-66-13-10-00', 'C0-56-27-18-69-65']
Empty DataFrame
Columns: []
Index: [C0-56-27-1B-F3-F4, C0-56-27-1B-9D-C5, C0-56-27-18-69-64, C0-56-27-18-69-66, C0-56-27-1B-F3-F5, C0-56-27-1B-9D-C6, D8-84-66-13-10-00, C0-56-27-18-69-65, loc]
(9, 0)
[-45 -45 -46 -47 -55 -57 -57 -61   0]
[-46 -47 -50 -55 -56 -56 -57   0   0]
[-44 -44 -46 -46 -50 -57 -58 -52   1]
[-42 -48 -41 -40 -52 -58 -53 -53   1]
[  0 -59 -45 -45 -47 -61 -48 -57   2]
[-43 -54 -48 -48 -49 -55 -54 -58   2]


In [6]:
## alldf now refers to all rssi values sorted by scan and AP(MAC Address) for all vectors
## We must now transpose this df, then export as a csv
alldf = alldf.T
## replace all zero values with nan (except)
alldf[reflist] = alldf[reflist].replace(0, np.nan)
        
print(alldf)
alldf.to_csv('../training_noavg.csv', index=False)

          C0-56-27-1B-F3-F4  C0-56-27-1B-9D-C5  C0-56-27-18-69-64  \
RSSI1                 -45.0              -45.0              -46.0   
RSSI2                   NaN                NaN                NaN   
RSSI3                 -46.0              -46.0              -54.0   
RSSI4                 -45.0              -45.0              -46.0   
RSSI5                 -50.0              -44.0              -46.0   
RSSI6                   NaN              -44.0              -46.0   
RSSI7                 -45.0              -45.0                NaN   
RSSI8                 -46.0              -46.0              -46.0   
RSSI9                 -47.0              -46.0              -46.0   
RSSI10                -45.0              -45.0              -47.0   
RSSI11                -47.0              -46.0              -47.0   
RSSI12                -45.0              -45.0              -44.0   
RSSI13                -46.0              -46.0              -46.0   
RSSI14                -45.0       